<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/02_clean_normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# 02_clean_normalize.ipynb
# Limpeza + Normalização + Parquet limpo
# =========================

import pandas as pd
import numpy as np

# 1) Preferir parquet validado (mais rápido). Se não existir, cai no CSV.
VALIDATED_PARQUET = PROCESSED_PATH / "trace_raw_validated.parquet"
CSV_FILE = RAW_PATH / "borg_traces_data.csv"

if VALIDATED_PARQUET.exists():
    df = pd.read_parquet(VALIDATED_PARQUET)
    log("Lido do parquet validado.")
else:
    df = pd.read_csv(CSV_FILE)
    df = df.drop(columns=["Unnamed: 0"], errors="ignore")
    df["time"] = pd.to_numeric(df["time"], errors="coerce")
    df.loc[df["time"] > 1e16, "time"] = np.nan
    log("Lido do CSV (fallback).")

# 2) Remover time NaN
df = df.dropna(subset=["time"]).copy()

# 3) Criar tempo relativo e hour (como nos seus dias 2 e 3)
t0 = df["time"].min()
df["t_rel_us"] = df["time"] - t0

# teto conservador: 3e12 us (~34 dias) usado por você no Dia 3 :contentReference[oaicite:7]{index=7}
df = df[(df["t_rel_us"] >= 0) & (df["t_rel_us"] <= 3.0e12)].copy()

df["hour"] = (df["t_rel_us"] // 3_600_000_000).astype("int64")

log(f"Horas (min..max): {df['hour'].min()}..{df['hour'].max()}")
log(f"Registros após limpeza: {len(df)}")

# 4) Remover hora 0 (artefato identificado no Dia 2) :contentReference[oaicite:8]{index=8}
before = len(df)
df = df[df["hour"] != 0].copy()
after = len(df)
log(f"Removida hour=0: {before-after} registros removidos.")

# 5) (Opcional) Dedupe se necessário — aqui só se fizer sentido pra você
# df = df.drop_duplicates()

# 6) Salvar dataset limpo definitivo
CLEAN_PARQUET = PROCESSED_PATH / "google_trace_clean.parquet"
df.to_parquet(CLEAN_PARQUET, index=False, compression="snappy")
log(f"Salvo: {CLEAN_PARQUET}")

df.head()